In [11]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import psycopg2 # needed to get database exception errors when uploading dataframe
import requests # package for getting data from the web
from sql_functions import get_engine


from datetime import datetime
import dataclasses
import json
import typing

import lxml.etree
import urllib3



In [10]:
max_url_length = 2000
max_id = 20000
stepsize_list = [x for x in range(0,max_id + 1,200)]

df = pd.DataFrame()
df_list = [df]

for idx in range(len(stepsize_list)-1):
    range_str = ""
    for i in range(stepsize_list[idx],stepsize_list[idx +1]):
        range_str += str(i) + ","
    range_str = range_str[:-1]
    url = f"http://boardgamegeek.com/xmlapi/boardgame/{range_str}"
    df = pd.concat([df,pd.read_xml(url)],ignore_index=True)

#df_list


In [ ]:
df.info()



In [ ]:
path = "data/"
filename = "boardgames_first_20000.xml"
xml_data = df.to_xml()
with open(f"{path}{filename}", 'w') as f:  # Writing in XML file
    for line in xml_data:
        f.write(line)

In [29]:
base_url = "https://boardgamegeek.com/xmlapi2/thing"

params = {
    'id' : 1,
    'type' : 'boardgame',
    'stats' : 1
}



In [30]:
status_set = {202, 408, 409, 413, 425, 429, 500, 502, 503, 504, 521, 598, 599}
retries = urllib3.util.retry.Retry(backoff_factor=60, status_forcelist=status_set)
http = urllib3.PoolManager(retries=retries)

parser = lxml.etree.XMLParser(remove_blank_text=True)

In [31]:
fields = {
    **params,
    'subtype': 'boardgame',
    'excludesubtype': 'boardgameexpansion'
}
resp = http.request('GET', base_url, fields=fields)

print('Reponse status:', resp.status)

with open('data/game_1.xml', 'wb') as fp:
    fp.write(resp.data)



Reponse status: 200


In [25]:
base_url = "https://boardgamegeek.com/xmlapi2/thing"
params = {'type' : 'boardgame', 'stats' : 1}
folder = "data/"

nums = np.arange(0,1001,200)
path = "data/boardgames.xml"

for idx in np.arange(len(nums)-1):
    rg = str(np.arange(nums[idx],nums[idx+1]))[:-1]
    fields = {**params, 'id' : rg, 'excludesubtype': 'boardgameexpansion'}
    resp = http.request('GET', base_url, fields=fields)
    print('Reponse status:', resp.status)
    filename = f"games_id_{nums[idx]}.xml"
    path = f"{folder}{filename}"
    with open(path, 'wb') as fp:
        fp.write(resp.data)

Reponse status: 200
Reponse status: 200
Reponse status: 200
Reponse status: 200
Reponse status: 200
